In [1]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_26495/1348678174.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Markdown


In [ ]:
!compressai-vision clean
!compressai-vision deregister --dataset-name quickstart-2-dummy --y
!compressai-vision deregister --dataset-name quickstart --y

The command line interface (cli) has all the functionality for evaluating your deep-learning compression algorithm against standardized benchmars.

The cli is accessed with the ``compressai-vision`` command that has several subcommands for handling datasets, evaluating your models with them and for generating plots.  In detail:

- ``compressai-vision -h`` gives you a short description of all commands
- ``compressai-vision manual`` shows you a more thorough description
- ``compressai-vision subcommand -h`` gives a detailed description of a certain subcommand

The very first subcommand you should try is ``info``.  It gives you information about the installed software stack, library versions and registered datasets:

In [4]:
!compressai-vision info


*** TORCH, CUDA, DETECTRON2, COMPRESSAI ***
torch version       : 1.9.1+cu102
cuda version        : 10.2
detectron2 version  : 0.6
compressai version  : 1.2.0.dev0

*** CHECKING GPU AVAILABILITY ***
device              : cpu

*** TESTING FFMPEG ***
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmp

Another basic command is ``list`` that just shows you the registered datasets:

In [2]:
!compressai-vision list

importing fiftyone
fiftyone imported

datasets currently registered into fiftyone
name, length, first sample path
mpeg-vcm-detection, 5000, /home/sampsa/fiftyone/mpeg-vcm-detection/data
mpeg-vcm-detection-dummy, 1, /home/sampsa/fiftyone/mpeg-vcm-detection/data
mpeg-vcm-segmentation, 5000, /home/sampsa/fiftyone/mpeg-vcm-segmentation/data
open-images-v6-validation, 8189, /home/sampsa/fiftyone/open-images-v6/validation/data
quickstart, 200, /home/sampsa/fiftyone/quickstart/data


Datasets can be registered to and deregistered from fiftyone using the ``register`` and ``deregister`` subcommands, and downloaded and registered directly from [fiftyone dataset zoo](https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html#dataset-zoo-quickstart) with the ``download`` command.  Let's use ``download`` to get the "quickstart" dataset:

In [14]:
!compressai-vision download --dataset-name=quickstart --y

importing fiftyone
fiftyone imported

Using list files:     None
Number of images:     ?
Database name   :     quickstart
Subname/split   :     None
Target dir      :     None

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Nice, we have ourselves a dataset to play with.  A note: the ``--y`` switch makes the command to run in non-interactive mode.  Let's take a closer look at the fields that the samples have in this datafield with ``show``:

In [15]:
!compressai-vision show --dataset-name=quickstart --y

importing fiftyone
fiftyone imported

Name:        quickstart
Media type:  image
Num samples: 200
Persistent:  True
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:   fiftyone.core.fields.FloatField
    predictions:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


Some fields of interests in each sample: ``filepath`` fields have the path to the downloaded images, while ``ground_truth`` fields have the ground-truth bounding boxes ("quickstart" dataset is a demo subset of COCO).

Next we'll crunch all the images in the dataset through a Detectron2 predictor and evaluate the results using the COCO evaluation protocol: as a result, we'll get a mAP accuracy for the Detectron2 model.  Note that we have to indicate the ground truth field with ``--gt-field=ground_truth``.  Option ``--slice=0:2`` takes only the first two samples from the dataset for this run: its only for debugging run, so please feel free to remove it.

In [22]:
!compressai-vision detectron2-eval --y --dataset-name=quickstart \
--slice=0:2 \
--gt-field=ground_truth \
--eval-method=coco \
--progressbar \
--output=detectron2_test.json \
--model=COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml

importing fiftyone
fiftyone imported
SURE YOU WANT THIS?

Using dataset          : quickstart
Dataset tmp clone      : detectron-run-sampsa-quickstart-2022-10-04-22-04-02-138278
Image scaling          : 100
Number of samples      : 2
Torch device           : cpu
Detectron2 model       : COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
Model was trained with : coco_2017_train
** Evaluation without Encoding/Decoding **
Ground truth data field name
                       : ground_truth
Eval. results will be saved to datafield
                       : detectron-predictions
Evaluation protocol    : coco
Progressbar            : True
Print progress         : 0
Output file            : detectron2_test.json
Peek model classes     :
['airplane', 'apple', 'backpack', 'banana', 'baseball bat'] ...
Peek dataset classes   :
['bird', 'horse', 'person'] ...
cloning dataset quickstart to detectron-run-sampsa-quickstart-2022-10-04-22-04-02-138278
instantiating Detectron2 predictor
/home/sampsa/silo/i

Let's see what we got:

In [23]:
!cat detectron2_test.json

{"dataset": "quickstart", "gt_field": "ground_truth", "tmp datasetname": "detectron-run-sampsa-quickstart-2022-10-04-22-04-02-138278", "slice": "0:2", "model": "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml", "compressai model": null, "custom model": null, "checkpoint": null, "vtm": false, "vtm_cache": null, "qpars": null, "bpp": [-1], "map": [0.5676567656765678], "map_per_class": [{"bird": 0.30297029702970296, "horse": 0.5, "person": 0.9}]}

Now we use again a Detectron2 predictor on our dataset.   However, before passing the images to Detectron2 model, they are first compressed and decompressed by using a pre-trained compressai model with a quality parameter 1 (``--qpars=1``).

We could evaluate for several quality parameters in serial by defining a list, i.e: ``--qpars=1,2,3`` and in parallel by launching the command separately for each particular value (say, for calculations in a queue/grid system).

A scaling can be applied on the images, as defined by the mpeg-vcm specifications (``--scale=100``).  Again, remember to remove ``--slice=0:2`` for an actual run.  

In [26]:
!compressai-vision detectron2-eval --y --dataset-name=quickstart \
--slice=0:2 \
--gt-field=ground_truth \
--eval-method=coco \
--scale=100 \
--progressbar \
--qpars=1 \
--compressai-model-name=bmshj2018_factorized \
--output=compressai_detectron2_test.json \
--model=COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml

importing fiftyone
fiftyone imported
SURE YOU WANT THIS?

Using dataset          : quickstart
Dataset tmp clone      : detectron-run-sampsa-quickstart-2022-10-04-22-17-06-604353
Image scaling          : 100
Number of samples      : 2
Torch device           : cpu
Detectron2 model       : COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
Model was trained with : coco_2017_train
Using compressai model : bmshj2018_factorized
Quality parameters     : [1]
Ground truth data field name
                       : ground_truth
Eval. results will be saved to datafield
                       : detectron-predictions
Evaluation protocol    : coco
Progressbar            : True
Print progress         : 0
Output file            : compressai_detectron2_test.json
Peek model classes     :
['airplane', 'apple', 'backpack', 'banana', 'baseball bat'] ...
Peek dataset classes   :
['bird', 'horse', 'person'] ...
cloning dataset quickstart to detectron-run-sampsa-quickstart-2022-10-04-22-17-06-604353
instantiati

Let's see what we got:

In [27]:
!cat compressai_detectron2_test.json

{"dataset": "quickstart", "gt_field": "ground_truth", "tmp datasetname": "detectron-run-sampsa-quickstart-2022-10-04-22-17-06-604353", "slice": "0:2", "model": "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml", "compressai model": "bmshj2018_factorized", "custom model": null, "checkpoint": null, "vtm": false, "vtm_cache": null, "qpars": [1], "bpp": [0.18178251121076233], "map": [0.44477447744774484], "map_per_class": [{"bird": 0.100990099009901, "horse": 0.3333333333333334, "person": 0.9}]}

Which is a single point on the mAP(bpp) curve